# Analyzing Toronto Neighborhoods

#### by Sushant Deshpande

### Let's start by importing libraries

In [10]:
from bs4 import BeautifulSoup
from collections import defaultdict
import requests
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Accessing the webpage
Let's access the wikipedia paage that lista all the postal codes for Toronto and save it in a pandas dataframe

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
title = soup.title
text = soup.get_text()
rows = soup.find_all('tr')
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

df = pd.DataFrame(list_rows)
df.head(10)

[<td align="center" style="border: 1px solid #FF0000; background-color: #FFE0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_A" title="List of postal codes of Canada: A">A</a>
</td>, <td align="center" style="border: 1px solid #FF4000; background-color: #FFE8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_B" title="List of postal codes of Canada: B">B</a>
</td>, <td align="center" style="border: 1px solid #FF8000; background-color: #FFF0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_C" title="List of postal codes of Canada: C">C</a>
</td>, <td align="center" style="border: 1px solid #FFC000; background-color: #FFF8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_E" title="List of postal codes of Canada: E">E</a>
</td>, <td align="center" style="border: 1px solid #FFFF00; background-color: #FFFFE0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_code

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


### Data cleanup

Split the dataframe and delete unwanted columns. Delete \n] and [

In [12]:
df.columns = ['title']
data1 = df['title'].str.split(',', expand=True)
data1.drop(data1.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]], axis=1, inplace=True)
data1.columns = ['PostalCode', 'Borough', 'Neighborhood']
data1.replace('\n','', regex=True)
data2 = data1.replace('\n]','', regex=True)
data2a = data1.PostalCode.str.replace("[", "")
data2.columns = ['PostalCode_1', 'Borough', 'Neighborhood']
data3 = pd.concat([data2a, data2], axis=1, sort=False)
data4 = data3.drop(['PostalCode_1'], axis=1)
data5 = data4.replace('Not assigned',np.nan, regex=True)
data6 = data5[pd.notnull(data5["Borough"])]
data6 = data6.reset_index(drop=True)
data6.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
data7a = data6['Neighborhood'].fillna(data6['Borough'])
data7 = pd.DataFrame(data7a)
data6.columns = ['PostalCode', 'Borough', 'Neighborhood_1']
data8a = pd.concat([data6, data7], axis=1, sort=False)
data8b = data8a.drop(['Neighborhood_1'], axis=1)
data8 = data8b[:212]
data9a = data8.groupby('PostalCode')['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
data9b = data8.groupby('PostalCode')['Borough'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
data9c = data9b['Borough'].str.split(', ').apply(set).str.join(', ')
data9d = pd.DataFrame(data9c)
data9d.head()
data9b.columns = ['PostalCode', 'Borough_1']
data9e = pd.concat([data9b, data9d], axis=1, sort=False)
data9f = data9e.drop(['Borough_1'], axis=1)
data9a.columns = ['PostalCode_1', 'Neighborhood']
data9g = pd.concat([data9f, data9a], axis=1, sort=False)
data9 = data9g.drop(['PostalCode_1'], axis=1)
data9.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


I downloaded the Geospatial_Coordinates.csv file and renamed the headers to PostalCode, Latitude, Longitude. We need to make sure that the datatypes of Latitude and Longitude are float. So I merged the imported csv file with my final dataframe on PostalCode i.e. merge the 2 dataframes in such a way that Postal code remains the same and the Latitude and Longitude for that PostalCode anr in corresponding cells.

In [14]:
cor_df = pd.read_csv("Geospatial_Coordinates.csv")
cor_df.columns = ['PostalCode', 'Latitude', 'Longitude']
data10 = pd.merge(data9, cor_df, on=['PostalCode'])
data10[['Latitude', 'Longitude']] = data10[['Latitude', 'Longitude']].astype(float)
data10.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of', address, 'are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, ON are 43.653963, -79.387207.


Now, let's analyze the east and west parts of Toronto. For this, we create a new data frames with Borough saying West Toronto and East Toronto.

In [16]:
data_west = data10[data10['Borough'].str.contains('West Toronto')].reset_index(drop=True)
data_west

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


In [17]:
data_east = data10[data10['Borough'].str.contains('East Toronto')].reset_index(drop=True)
data_east

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558


Now, let's visualize it on a map.

In [ ]:
# create map of Toronto using latitude and longitude values
map_west = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_west['Latitude'],
                                           data_west['Longitude'],
                                           data_west['Borough'],
                                           data_west['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_west)  
    
map_west

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-1.jpg"> </a>

In [ ]:
# create map of Toronto using latitude and longitude values
map_east = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_east['Latitude'],
                                           data_east['Longitude'],
                                           data_east['Borough'],
                                           data_east['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_east)  
    
map_east

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-2.jpg"> </a>

#### Now, let's leverage the power of FourSquare!

In [59]:
# @hidden_cell
CLIENT_ID = 'HIDDEN' # your Foursquare ID
CLIENT_SECRET = 'HIDDEN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HIDDEN
CLIENT_SECRET:HIDDEN


Let's take the first cell from each cell and see analyze it.

In [21]:
data_east.loc[0, 'Neighborhood']

' The Beaches'

In [22]:
data_west.loc[0, 'Neighborhood']

' Dovercourt Village,  Dufferin'

In [23]:
neighborhood_latitude_east = data_east.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_east = data_east.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name_east = data_east.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_east, 
                                                               neighborhood_latitude_east, 
                                                               neighborhood_longitude_east))

Latitude and longitude values of  The Beaches are 43.67635739999999, -79.2930312.


In [24]:
neighborhood_latitude_west = data_west.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_west = data_west.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name_west = data_west.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_west, 
                                                               neighborhood_latitude_west, 
                                                               neighborhood_longitude_west))

Latitude and longitude values of  Dovercourt Village,  Dufferin are 43.66900510000001, -79.4422593.


In [25]:
LIMIT = 100
radius = 1000

url_4s_east = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_east, 
    neighborhood_longitude_east, 
    radius, 
    LIMIT)
url_4s_east

'https://api.foursquare.com/v2/venues/explore?&client_id=HIDDEN&client_secret=HIDDEN&v=20180604&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [26]:
LIMIT = 100
radius = 1000

url_4s_west = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_west, 
    neighborhood_longitude_west, 
    radius, 
    LIMIT)
url_4s_west

'https://api.foursquare.com/v2/venues/explore?&client_id=HIDDEN&client_secret=HIDDEN&v=20180604&ll=43.66900510000001,-79.4422593&radius=1000&limit=100'

In [27]:
results_east = requests.get(url_4s_east).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues_east = results_east['response']['groups'][0]['items']
nearby_venues_east = json_normalize(venues_east) # flatten JSON
# filter columns
filtered_columns_east = ['venue.name',
                    'venue.categories',
                    'venue.location.lat',
                    'venue.location.lng']
nearby_venues_east =nearby_venues_east.loc[:, filtered_columns_east]
# filter the category for each row
nearby_venues_east['venue.categories'] = nearby_venues_east.apply(get_category_type, axis=1)
# clean columns
nearby_venues_east.columns = [col.split(".")[-1] for col in nearby_venues_east.columns]
nearby_venues_east.head()

,name,categories,lat,lng
0,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
3,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993
4,The Beech Tree,Gastropub,43.680493,-79.288846


In [28]:
results_west = requests.get(url_4s_west).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues_west = results_west['response']['groups'][0]['items']
nearby_venues_west = json_normalize(venues_west) # flatten JSON
# filter columns
filtered_columns_west = ['venue.name',
                    'venue.categories',
                    'venue.location.lat',
                    'venue.location.lng']
nearby_venues_west =nearby_venues_west.loc[:, filtered_columns_west]
# filter the category for each row
nearby_venues_west['venue.categories'] = nearby_venues_west.apply(get_category_type, axis=1)
# clean columns
nearby_venues_west.columns = [col.split(".")[-1] for col in nearby_venues_west.columns]
nearby_venues_west.head()

,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Blood Brothers Brewing,Brewery,43.669944,-79.436533
3,Planet Fitness Toronto Galleria,Gym / Fitness Center,43.667588,-79.442574
4,Happy Bakery & Pastries,Bakery,43.667050,-79.441791


In [29]:
nearby_venues_west.shape

(70, 4)

In [30]:
print('East Toronto has {} venues,'.format(nearby_venues_east.shape[0]), 'and West Toronto has {} venues.'.format(nearby_venues_west.shape[0]))

East Toronto has 85 venues, and West Toronto has 70 venues.


As we can see above, East Toronto has 85 venues, and West Toronto has 70 venues.

In [31]:
def getNearbyVenues_east(names, latitudes, longitudes, radius=1000):
    
    venues_list_east=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url_4s1_east = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results_east = requests.get(url_4s1_east).json()["response"]['groups'][0]['items']

        venues_list_east.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_east])

    nearby_venues_east = pd.DataFrame([item for venues_list_east in venues_list_east for item in venues_list_east])
    nearby_venues_east.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_east)

In [32]:
def getNearbyVenues_west(names, latitudes, longitudes, radius=1000):
    
    venues_list_west=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url_4s1_west = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results_west = requests.get(url_4s1_west).json()["response"]['groups'][0]['items']

        venues_list_west.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_west])

    nearby_venues_west = pd.DataFrame([item for venues_list_west in venues_list_west for item in venues_list_west])
    nearby_venues_west.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_west)

In [33]:
toronto_venues_east = getNearbyVenues_east(names=data_east['Neighborhood'],
                                   latitudes=data_east['Latitude'],
                                   longitudes=data_east['Longitude']
                                  )

 The Beaches
 The Danforth West,  Riverdale
 The Beaches West,  India Bazaar
 Studio District
 Business reply mail Processing Centre969 Eastern


In [34]:
toronto_venues_west = getNearbyVenues_west(names=data_west['Neighborhood'],
                                   latitudes=data_west['Latitude'],
                                   longitudes=data_west['Longitude']
                                  )

 Dovercourt Village,  Dufferin
 Little Portugal,  Trinity
 Brockton,  Exhibition Place,  Parkdale Village
 High Park,  The Junction South
 Parkdale,  Roncesvalles
 Runnymede,  Swansea


In [35]:
toronto_venues_east.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


In [36]:
toronto_venues_west.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dovercourt Village, Dufferin",43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,"Dovercourt Village, Dufferin",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,"Dovercourt Village, Dufferin",43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
3,"Dovercourt Village, Dufferin",43.669005,-79.442259,Planet Fitness Toronto Galleria,43.667588,-79.442574,Gym / Fitness Center
4,"Dovercourt Village, Dufferin",43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery


In [37]:
toronto_venues_east.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues_east['Venue Category'].unique())))

There are 133 uniques categories.


In [38]:
toronto_venues_west.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues_west['Venue Category'].unique())))

There are 162 uniques categories.


In [39]:
# one hot encoding
toronto_onehot_east = pd.get_dummies(toronto_venues_east[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot_east['Neighborhood'] = toronto_venues_east['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns_east = [toronto_onehot_east.columns[-1]] + list(toronto_onehot_east.columns[:-1])
toronto_onehot_east = toronto_onehot_east[fixed_columns_east]

toronto_onehot_east.head()

,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Store,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Store,Nail Salon,Neighborhood,New American Restaurant,Pakistani Restaurant,Park,Pet Store,Pharmacy,Pide Place,Pizza Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Spa,Sporting Goods Shop,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
# one hot encoding
toronto_onehot_west = pd.get_dummies(toronto_venues_west[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot_west['Neighborhood'] = toronto_venues_west['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns_west = [toronto_onehot_west.columns[-1]] + list(toronto_onehot_west.columns[:-1])
toronto_onehot_west = toronto_onehot_west[fixed_columns_west]

toronto_onehot_west.head()

,Zoo,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beach Bar,Beer Bar,Beer Store,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Lab,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Pool,Portuguese Restaurant,Poutine Place,Pub,Record Shop,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Stadium,Supermarket,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Dovercourt Village, Dufferin",0,0,0,0,0,0,0,0,0

In [41]:
toronto_grouped_east = toronto_onehot_east.groupby('Neighborhood').mean().reset_index()
num_top_venues_east = 5

for hood in toronto_grouped_east['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_east[toronto_grouped_east['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_east))
    print('\n')

---- Business reply mail Processing Centre969 Eastern----
                venue  freq
0                Park  0.11
1         Coffee Shop  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.04
4       Burrito Place  0.04


---- Studio District----
                 venue  freq
0          Coffee Shop  0.07
1                  Bar  0.06
2               Bakery  0.05
3  American Restaurant  0.05
4                 Café  0.05


---- The Beaches----
            venue  freq
0     Coffee Shop  0.08
1             Pub  0.07
2     Pizza Place  0.06
3          Bakery  0.04
4  Breakfast Spot  0.04


---- The Beaches West,  India Bazaar----
               venue  freq
0  Indian Restaurant  0.09
1        Coffee Shop  0.05
2              Beach  0.04
3     Sandwich Place  0.04
4               Café  0.04


---- The Danforth West,  Riverdale----
                  venue  freq
0      Greek Restaurant  0.13
1           Coffee Shop  0.07
2                   Pub  0.05
3                  Café  0.05
4  Fast Fo

In [42]:
toronto_grouped_west = toronto_onehot_west.groupby('Neighborhood').mean().reset_index()
num_top_venues_west = 5

for hood in toronto_grouped_west['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_west[toronto_grouped_west['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_west))
    print('\n')

---- Brockton,  Exhibition Place,  Parkdale Village----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.05
2              Restaurant  0.04
3                     Bar  0.04
4  Furniture / Home Store  0.03


---- Dovercourt Village,  Dufferin----
         venue  freq
0         Café  0.10
1         Park  0.07
2  Coffee Shop  0.07
3  Supermarket  0.04
4       Bakery  0.04


---- High Park,  The Junction South----
                venue  freq
0                 Bar  0.09
1                Café  0.08
2         Coffee Shop  0.06
3   Convenience Store  0.04
4  Italian Restaurant  0.04


---- Little Portugal,  Trinity----
          venue  freq
0          Café  0.08
1           Bar  0.08
2    Restaurant  0.05
3        Bakery  0.05
4  Cocktail Bar  0.04


---- Parkdale,  Roncesvalles----
            venue  freq
0     Coffee Shop  0.05
1             Bar  0.05
2            Café  0.04
3  Breakfast Spot  0.04
4     Pizza Place  0.04


---- Runnymede,  Swansea-

### Now, let's a get a list of top 10 venues in east and west Toronto

In [43]:
def return_most_common_venues_east(row, num_top_venues_east):
    row_categories_east = row.iloc[1:]
    row_categories_sorted_east = row_categories_east.sort_values(ascending=False)
    
    return row_categories_sorted_east.index.values[0:num_top_venues_east]

In [44]:
def return_most_common_venues_west(row, num_top_venues_west):
    row_categories_west = row.iloc[1:]
    row_categories_sorted_west = row_categories_west.sort_values(ascending=False)
    
    return row_categories_sorted_west.index.values[0:num_top_venues_west]

In [45]:
num_top_venues_east = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues_east):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_east = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_east['Neighborhood'] = toronto_grouped_east['Neighborhood']

for ind in np.arange(toronto_grouped_east.shape[0]):
    neighborhoods_venues_sorted_east.iloc[ind, 1:] = return_most_common_venues_east(toronto_grouped_east.iloc[ind, :], num_top_venues_east)

neighborhoods_venues_sorted_east

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre969 Eastern,Park,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Burrito Place,Brewery,Thai Restaurant,Breakfast Spot,Comic Shop
1,Studio District,Coffee Shop,Bar,Café,Bakery,American Restaurant,Italian Restaurant,Vietnamese Restaurant,Diner,Brewery,Sandwich Place
2,The Beaches,Coffee Shop,Pub,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Bakery,Burger Joint,Indian Restaurant
3,"The Beaches West, India Bazaar",Indian Restaurant,Coffee Shop,Sandwich Place,Beach,Café,Bakery,Italian Restaurant,Burrito Place,Spa,Restaurant
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Spa,Sandwich Place,Diner


In [46]:
num_top_venues_west = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues_west):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_west = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_west['Neighborhood'] = toronto_grouped_west['Neighborhood']

for ind in np.arange(toronto_grouped_west.shape[0]):
    neighborhoods_venues_sorted_west.iloc[ind, 1:] = return_most_common_venues_west(toronto_grouped_west.iloc[ind, :], num_top_venues_west)

neighborhoods_venues_sorted_west

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Restaurant,Bar,Furniture / Home Store,Vegetarian / Vegan Restaurant,Bakery,Soccer Stadium,Sandwich Place,Gift Shop
1,"Dovercourt Village, Dufferin",Café,Park,Coffee Shop,Supermarket,Sushi Restaurant,Bar,Bakery,Breakfast Spot,Convenience Store,Pharmacy
2,"High Park, The Junction South",Bar,Café,Coffee Shop,Italian Restaurant,Convenience Store,Thai Restaurant,Sandwich Place,Sushi Restaurant,Antique Shop,Bakery
3,"Little Portugal, Trinity",Bar,Café,Restaurant,Bakery,Coffee Shop,Cocktail Bar,Men's Store,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant
4,"Parkdale, Roncesvalles",Bar,Coffee Shop,Sushi Restaurant,Breakfast Spot,Pizza Place,Café,Thai Restaurant,Gym,Eastern European Restaurant,Bakery
5,"Runnymede, Swansea",Coffee Shop,Café,Bakery,Pizza Place,Italian Restaurant,Park,Pub,Spa,Gastropub,Diner


In [47]:
kclusters = 5

toronto_grouped_clustering_east = toronto_grouped_east.drop('Neighborhood', 1)

kmeans_east = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering_east)

kmeans_east.labels_[0:10]

array([3, 2, 1, 4, 0], dtype=int32)

In [48]:
kclusters = 5

toronto_grouped_clustering_west = toronto_grouped_west.drop('Neighborhood', 1)

kmeans_west = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering_west)

kmeans_west.labels_[0:10]

array([1, 3, 4, 1, 0, 2], dtype=int32)

In [49]:
toronto_merged_east = data_east

toronto_merged_east['Cluster Labels'] = kmeans_east.labels_

toronto_merged_east = toronto_merged_east.join(neighborhoods_venues_sorted_east.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_east.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Coffee Shop,Pub,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Bakery,Burger Joint,Indian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Spa,Sandwich Place,Diner
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Indian Restaurant,Coffee Shop,Sandwich Place,Beach,Café,Bakery,Italian Restaurant,Burrito Place,Spa,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,Bar,Café,Bakery,American Restaurant,Italian Restaurant,Vietnamese Restaurant,Diner,Brewery,Sandwich Place
4,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,0,Park,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Burrito Place,Brewery,Thai Restaurant,Breakfast Spot,Comic Shop


In [50]:
toronto_merged_west = data_west

toronto_merged_west['Cluster Labels'] = kmeans_west.labels_

toronto_merged_west = toronto_merged_west.join(neighborhoods_venues_sorted_west.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_west.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,1,Café,Park,Coffee Shop,Supermarket,Sushi Restaurant,Bar,Bakery,Breakfast Spot,Convenience Store,Pharmacy
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,3,Bar,Café,Restaurant,Bakery,Coffee Shop,Cocktail Bar,Men's Store,Italian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,4,Café,Coffee Shop,Restaurant,Bar,Furniture / Home Store,Vegetarian / Vegan Restaurant,Bakery,Soccer Stadium,Sandwich Place,Gift Shop
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,1,Bar,Café,Coffee Shop,Italian Restaurant,Convenience Store,Thai Restaurant,Sandwich Place,Sushi Restaurant,Antique Shop,Bakery
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Bar,Coffee Shop,Sushi Restaurant,Breakfast Spot,Pizza Place,Café,Thai Restaurant,Gym,Eastern European Restaurant,Bakery


In [ ]:
map_clusters_east = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_east['Latitude'],
                                  toronto_merged_east['Longitude'],
                                  toronto_merged_east['Neighborhood'],
                                  toronto_merged_east['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_east)
       
map_clusters_east

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-3.jpg"> </a>

In [ ]:
map_clusters_west = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_west['Latitude'],
                                  toronto_merged_west['Longitude'],
                                  toronto_merged_west['Neighborhood'],
                                  toronto_merged_west['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_west)
       
map_clusters_west

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-4.jpg"> </a>

In [53]:
toronto_east_group = toronto_venues_east.groupby('Venue Category').count()
print('There are {} uniques categories.'.format(len(toronto_venues_east['Venue Category'].unique())))

There are 133 uniques categories.


In [54]:
toronto_west_group = toronto_venues_west.groupby('Venue Category').count()
print('There are {} uniques categories.'.format(len(toronto_venues_west['Venue Category'].unique())))

There are 162 uniques categories.


In [55]:
toronto_east_group.sort_values(by='Neighborhood', ascending=True, inplace=True)

df_top10_east = toronto_east_group['Neighborhood'].tail(10)
df_top10_east

Venue Category
Indian Restaurant     10
Italian Restaurant    10
Bar                   11
Park                  12
Pub                   13
Bakery                13
Pizza Place           13
Greek Restaurant      14
Café                  14
Coffee Shop           29
Name: Neighborhood, dtype: int64

In [56]:
toronto_west_group.sort_values(by='Neighborhood', ascending=True, inplace=True)

df_top10_west = toronto_west_group['Neighborhood'].tail(10)
df_top10_west

Venue Category
Breakfast Spot        10
Sushi Restaurant      12
Park                  12
Restaurant            13
Pizza Place           13
Italian Restaurant    14
Bakery                21
Bar                   30
Coffee Shop           33
Café                  41
Name: Neighborhood, dtype: int64

#### Let’s try to plot them using bar graph to make them more visually appealing and easy to understand.

In [ ]:
df_top10_east.plot(kind='barh', figsize=(12, 12), color='limegreen')
plt.xlabel('Kind of Shops')
plt.title('Top 15 kind of shops in Totonto - East')

plt.show()

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-5.jpg"> </a>

In [ ]:
df_top10_west.plot(kind='barh', figsize=(12, 12), color='orange')
plt.xlabel('Kind of Shops')
plt.title('Top 15 kind of shops in Totonto - West')

plt.show()

<img src = "https://raw.githubusercontent.com/sushant822/Analyzing-Toronto-Neighborhoods/master/c-f-6.jpg"> </a>